# VacationPy
----


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
humidity_pd = pd.read_csv("../WeatherPy/weather_data.csv")
humidity_pd.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,zhovtneve,50.6637,24.2492,67.03,69,9,5.55,UA,1626040070
1,bluff,-46.6000,168.3333,43.84,83,100,15.41,NZ,1626040070
2,luderitz,-26.6481,15.1594,54.45,83,12,4.99,NaN,1626040071
3,sivas,39.7916,37.0019,59.63,94,0,3.44,TR,1626040071
4,mar del plata,-38.0023,-57.5575,55.87,57,0,10.00,AR,1626039887


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
# Store latitude and longitude in locations
locations = humidity_pd[["Latitude", "Longitude"]]
max_humid = humidity_pd["Humidity"].max()
max_humid 

humid = humidity_pd["Humidity"]

In [7]:
#Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=max_humid,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#setting index to be able to use loc function for narrowing down the cities with specified weather conditions, dropping null values
weather_index_pd = humidity_pd.set_index(["City"]).dropna()
weather_index_pd

,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City,,,,,,,,
zhovtneve,50.6637,24.2492,67.03,69,9,5.55,UA,1626040070
bluff,-46.6000,168.3333,43.84,83,100,15.41,NZ,1626040070
sivas,39.7916,37.0019,59.63,94,0,3.44,TR,1626040071
mar del plata,-38.0023,-57.5575,55.87,57,0,10.00,AR,1626039887
kaka,37.3482,59.6143,82.49,29,5,0.45,TM,1626040071
...,...,...,...,...,...,...,...,...
darnah,32.7670,22.6367,77.68,73,0,14.99,LY,1626040207
aswan,24.0934,32.9070,91.60,23,17,8.28,EG,1626040208
nongan,44.3820,125.1685,75.61,88,72,8.30,CN,1626040208


### Weather conditions to narrow down the cities:
* max temperature more than 77 degrees and less than 86 degrees
* cloudiness less than 10%
* wind speed less than 10 mph


In [10]:
weather_criteria_df = weather_index_pd.loc[(weather_index_pd["Max Temp"]>77) & (weather_index_pd["Max Temp"]<86)&(weather_index_pd["Cloudiness"]<10)&(weather_index_pd["Wind Speed"]<10)] 
weather_criteria_df

,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City,,,,,,,,
kaka,37.3482,59.6143,82.49,29,5,0.45,TM,1626040071
syracuse,37.0881,15.2792,77.90,71,0,7.09,IT,1626040095
gat,31.6100,34.7642,81.39,100,0,1.68,IL,1626040112
mankato,44.1636,-93.9994,84.49,48,1,8.05,US,1626040017
makat,47.6469,53.3463,79.68,47,9,8.08,KZ,1626040161
vercelli,45.3231,8.4153,77.86,68,0,1.81,IT,1626040163
erzin,36.9559,36.2000,79.52,76,0,1.01,TR,1626040166
fort-shevchenko,44.5086,50.2630,78.84,55,0,7.29,KZ,1626040180
akdepe,42.0551,59.3788,78.80,21,0,6.64,TM,1626040184


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
#reseting index
weather_criteria_df = weather_criteria_df.reset_index()
weather_criteria_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kaka,37.3482,59.6143,82.49,29,5,0.45,TM,1626040071
1,syracuse,37.0881,15.2792,77.90,71,0,7.09,IT,1626040095
2,gat,31.6100,34.7642,81.39,100,0,1.68,IL,1626040112
3,mankato,44.1636,-93.9994,84.49,48,1,8.05,US,1626040017
4,makat,47.6469,53.3463,79.68,47,9,8.08,KZ,1626040161
5,vercelli,45.3231,8.4153,77.86,68,0,1.81,IT,1626040163
6,erzin,36.9559,36.2000,79.52,76,0,1.01,TR,1626040166
7,fort-shevchenko,44.5086,50.2630,78.84,55,0,7.29,KZ,1626040180
8,akdepe,42.0551,59.3788,78.80,21,0,6.64,TM,1626040184
9,junction city,39.0286,-96.8314,82.62,47,1,3.00,US,1626040205


In [12]:
#creating new DataFrame and adding new column for "Hotel Name"
hotel_df = pd.DataFrame(weather_criteria_df[["Latitude","Longitude","City","Country"]])
hotel_df["Hotel Name"] = "" 
hotel_df


,Latitude,Longitude,City,Country,Hotel Name
0,37.3482,59.6143,kaka,TM,
1,37.0881,15.2792,syracuse,IT,
2,31.6100,34.7642,gat,IL,
3,44.1636,-93.9994,mankato,US,
4,47.6469,53.3463,makat,KZ,
5,45.3231,8.4153,vercelli,IT,
6,36.9559,36.2000,erzin,TR,
7,44.5086,50.2630,fort-shevchenko,KZ,
8,42.0551,59.3788,akdepe,TM,
9,39.0286,-96.8314,junction city,US,


In [13]:
# iterate through filtered locations 

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the search term for nearby hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Use the latitude, longitude to identify city
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # changing location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
   
    # making request 
    name_url = requests.get(base_url, params=params)
 
    # converting to json
    name_url = name_url.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # for any missing data skipping
    try:
        print(name_url["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"] = name_url["results"][0]["name"]
       
    except (KeyError, IndexError):
        print("No hotel/result found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = "- no hotel found - "

No hotel/result found... skipping.
Minareto
Desert Gat
Mankato City Center Hotel
Gostinitsa Daulet
La Terrazza Vercelli Bed & Charme
Hattusa Vacation Thermal Club Erzin
Chagala Bautino Hotel
No hotel/result found... skipping.
Courtyard by Marriott Junction City


In [14]:
# cities with nearby hotel found in DataFrame
hotel_df


,Latitude,Longitude,City,Country,Hotel Name
0,37.3482,59.6143,kaka,TM,- no hotel found -
1,37.0881,15.2792,syracuse,IT,Minareto
2,31.6100,34.7642,gat,IL,Desert Gat
3,44.1636,-93.9994,mankato,US,Mankato City Center Hotel
4,47.6469,53.3463,makat,KZ,Gostinitsa Daulet
5,45.3231,8.4153,vercelli,IT,La Terrazza Vercelli Bed & Charme
6,36.9559,36.2000,erzin,TR,Hattusa Vacation Thermal Club Erzin
7,44.5086,50.2630,fort-shevchenko,KZ,Chagala Bautino Hotel
8,42.0551,59.3788,akdepe,TM,- no hotel found -
9,39.0286,-96.8314,junction city,US,Courtyard by Marriott Junction City


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Adding marker layer ontop of heat map, displaying info labels for each pin 
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))